# Saraga Dataset (Three Stem Conversion)

In [1]:
import librosa as lb
import soundfile as sf
import numpy as np
import pandas as pd
import os
import shutil
from tqdm import tqdm

In [2]:
path = "/home/rajesh/Desktop/Datasets/saraga1.5_carnatic/"
top = sorted([i for i in os.listdir(path) if os.path.isdir(path+i)])
print("Top folders: ", len(top))
top_path = sorted([path+i for i in os.listdir(path) if os.path.isdir(path+i)])

Top folders:  26


In [3]:
mid_folders = []
len_sub_folders = []

mid_folders_path = []

for i in top_path:
    sub_folders = sorted([j for j in os.listdir(i) if os.path.isdir(os.path.join(i, j))])
    len_sub_folders.append(len(sub_folders))
    mid_folders.append(sub_folders)
    sub_folders_path = sorted([os.path.join(i, j) for j in os.listdir(i) if os.path.isdir(os.path.join(i, j))])
    mid_folders_path.append(sub_folders_path)

In [4]:
print('Total folders: ', sum(len_sub_folders))

Total folders:  249


In [5]:
files = []
files_path = []
for k in range(len(top)):
    for i in mid_folders_path[k]:
        file = sorted([j for j in os.listdir(i) if j[-3:] == 'mp3'])
        files.append(file)
        
        file_p = sorted([os.path.join(i, j) for j in os.listdir(i) if j[-3:] == 'mp3'])
        files_path.append(file_p)

In [6]:
len(files)

249

In [7]:
dest_folder = "/home/rajesh/Desktop/Datasets/Saraga3stem/"
os.makedirs(dest_folder, exist_ok=True)

In [8]:
c = 1
for i in tqdm(range(len(files))):
    if len(files[i]) >= 3:
        folder_name = files[i][0].split('.')[0].replace(" ", "")
        os.makedirs(dest_folder+folder_name, exist_ok=True)
        for j in range(len(files[i])):
            name = files[i][j].split('.')
            if name[1] == 'mp3':
                shutil.copy2(files_path[i][j], os.path.join(dest_folder+folder_name, 'mixture.mp3'))
            else:
                deep_name = name[1].split('-')
                if deep_name[1] == "ghatam":
                    shutil.copy2(files_path[i][j], os.path.join(dest_folder+folder_name, 'ghatam.mp3'))
                if deep_name[1] == "violin":
                    shutil.copy2(files_path[i][j], os.path.join(dest_folder+folder_name, 'violin.mp3'))
                if deep_name[1] == "vocal":
                    try:
                        if deep_name[2] == 's':
                            shutil.copy2(files_path[i][j], os.path.join(dest_folder+folder_name, 'vocals.mp3'))
                        else:
                            shutil.copy2(files_path[i][j], os.path.join(dest_folder+folder_name, 'svocals.mp3'))
                    except:
                        shutil.copy2(files_path[i][j], os.path.join(dest_folder+folder_name, 'vocals.mp3'))
                if deep_name[1] == "mridangam":
                    if deep_name[2] == "right":
                        shutil.copy2(files_path[i][j], os.path.join(dest_folder+folder_name, 'right.mp3'))
                    else:
                        shutil.copy2(files_path[i][j], os.path.join(dest_folder+folder_name, 'left.mp3'))
        
        #print(c, "folders done")            
        c+=1
            
print(c)

100%|█████████████████████████████████████████| 249/249 [07:36<00:00,  1.83s/it]

169


In [9]:
c = 1
s_v = 0
for i in tqdm(range(len(files))):
    if len(files[i]) >= 3:
        folder_name = files[i][0].split('.')[0].replace(" ", "")
        os.makedirs(dest_folder+folder_name, exist_ok=True)
        for j in range(len(files[i])):
            name = files[i][j].split('.')
            if name[1] == 'mp3':
                pass
            else:
                deep_name = name[1].split('-')
                if deep_name[1] == "vocal":
                    if len(deep_name) == 3:
                        shutil.copy2(files_path[i][j], os.path.join(dest_folder+folder_name, 'svocals.mp3'))
                        s_v += 1
                    else:
                        shutil.copy2(files_path[i][j], os.path.join(dest_folder+folder_name, 'vocals.mp3'))
        
        #print(c, "folders done")            
        c+=1
            
print(c)
print(s_v, 'Secondary vocal copied!')

100%|█████████████████████████████████████████| 249/249 [01:26<00:00,  2.88it/s]

169
24 Secondary vocal copied!


### Converting copied .mp3 to .wav

In [10]:
from pydub import AudioSegment

In [11]:
dest_folder = "/home/rajesh/Desktop/Datasets/Saraga3stem/"
files = [dest_folder+i for i in os.listdir(dest_folder)]
c = 1
for i in tqdm(files):
    file = os.listdir(i)
    for j in file:
        #print(i+'/'+j)
        sound = AudioSegment.from_mp3(i+'/'+j)
        sound.export(i+'/'+j[:-4]+'.wav', format="wav")
    #print('{}/161 folders done'.format(c))
    c+=1

100%|█████████████████████████████████████████| 161/161 [23:17<00:00,  8.68s/it]


### Deleting .mp3

In [12]:
dest_folder = "/home/rajesh/Desktop/Datasets/Saraga3stem/"
files = [dest_folder+i for i in os.listdir(dest_folder)]
for i in tqdm(files):
    file = os.listdir(i)
    for j in file:
        if j[-3:] == 'mp3':
            os.remove(i+'/'+j)

100%|█████████████████████████████████████████| 161/161 [00:35<00:00,  4.53it/s]


### Vocal & Vocal Secondary Mix:

In [13]:
dest_folder = "/home/rajesh/Desktop/Datasets/Saraga3stem/"
files = [dest_folder+i for i in os.listdir(dest_folder)]

for i in tqdm(files):
    file = os.listdir(i)
    if 'vocals.wav' in file and 'svocals.wav' in file:
        vocal1 = AudioSegment.from_file(i+'/vocals.wav', format="wav")
        vocal2 = AudioSegment.from_file(i+'/svocals.wav', format="wav")
        out = vocal1.overlay(vocal2, position=0)
        os.remove(i+'/vocals.wav')
        os.remove(i+'/svocals.wav')
        out.export(i+'/vocals.wav', format="wav")

100%|█████████████████████████████████████████| 161/161 [00:46<00:00,  3.45it/s]


### Mirdangam Left & Right Mix

In [14]:
dest_folder = "/home/rajesh/Desktop/Datasets/Saraga3stem/"
files = [dest_folder+i for i in os.listdir(dest_folder)]

for i in tqdm(files):
    file = os.listdir(i)
    if 'left.wav' in file and 'right.wav' in file:
        left = AudioSegment.from_file(i+'/left.wav', format="wav")
        right = AudioSegment.from_file(i+'/right.wav', format="wav")
        out = left.overlay(right, position=0)
        os.remove(i+'/left.wav')
        os.remove(i+'/right.wav')
        out.export(i+'/mridangam.wav', format="wav")

100%|█████████████████████████████████████████| 161/161 [06:48<00:00,  2.54s/it]


### Counting only ghatam

In [15]:
dest_folder = "/home/rajesh/Desktop/Datasets/Saraga3stem/"
files = [dest_folder+i for i in os.listdir(dest_folder)]

c = 0
for i in tqdm(files):
    file = os.listdir(i)
    if 'ghatam.wav' in file:
        c+=1
print(c, 'files available')

100%|█████████████████████████████████████████| 161/161 [00:05<00:00, 31.59it/s]

45 files available


### Counting Valid Three Stems (Vocals, Violin, Mridangam, Mixed)

In [16]:
dest_folder = "/home/rajesh/Desktop/Datasets/Saraga3stem/"
files = [dest_folder+i for i in os.listdir(dest_folder)]

c = 0
for i in tqdm(files):
    file = os.listdir(i)
    if 'vocals.wav' in file and 'violin.wav' in file and 'mridangam.wav' in file and 'mixture.wav' in file:
        c+=1
print(c, 'files available')

100%|██████████████████████████████████████| 161/161 [00:00<00:00, 15720.71it/s]

161 files available


### Converting Mixture from 2 channel to single channel

In [6]:
dest_folder = "/home/rajesh/Desktop/Datasets/Saraga3stem/"
files = [dest_folder+i for i in os.listdir(dest_folder)]

skip = 0
for i in tqdm(files):
    file = os.listdir(i)
    if 'vocals.wav' in file and 'violin.wav' in file and 'mridangam.wav' in file and 'mixture.wav' in file:
        #print(i+'/vocals.wav')
        #vocal, vsr = sf.read(i+'/vocals.wav')
        #violin, visr = sf.read(i+'/violin.wav')
        #mridangam, mrsr = sf.read(i+'/mridangam.wav')
        mixture, msr = sf.read(i+'/mixture.wav')
        
        avg = []
        try:
            for j in mixture:
                avg.append((j[0] + j[1])/2)
                skip+=1 
            flag = True
        except:
            avg = mixture
            flag = False
            
        if flag == True:
            print("No")
            #os.remove(i+'/mixture.wav')
            #sf.write(i+'/mixture.wav', avg, msr)
        
print(skip, ' files skipped')

100%|█████████████████████████████████████████| 161/161 [01:57<00:00,  1.37it/s]

0  files skipped


### Making all of same length

In [9]:
dest_folder = "/home/rajesh/Desktop/Datasets/Saraga3stem/"
files = [dest_folder+i for i in os.listdir(dest_folder)]

for i in tqdm(files):
    file = os.listdir(i)
    if 'vocals.wav' in file and 'violin.wav' in file and 'mridangam.wav' in file and 'mixture.wav' in file:
        vocal, vsr = sf.read(i+'/vocals.wav')
        violin, visr = sf.read(i+'/violin.wav')
        mridangam, mrsr = sf.read(i+'/mridangam.wav')
        mixture, msr = sf.read(i+'/mixture.wav')
        
        
        low = min([len(vocal), len(violin), len(mridangam), len(mixture)])
        
        mixture = mixture[:low]
        
        os.remove(i+'/mixture.wav')
        sf.write(i+'/mixture.wav', avg, msr)

100%|█████████████████████████████████████████| 161/161 [22:02<00:00,  8.21s/it]


In [10]:
dest_folder = "/home/rajesh/Desktop/Datasets/Saraga3stem/"
files = [dest_folder+i for i in os.listdir(dest_folder)]

for i in tqdm(files):
    file = os.listdir(i)
    if 'vocals.wav' in file and 'violin.wav' in file and 'mridangam.wav' in file and 'mixture.wav' in file:
        vocal, vsr = sf.read(i+'/vocals.wav')
        violin, visr = sf.read(i+'/violin.wav')
        mridangam, mrsr = sf.read(i+'/mridangam.wav')
        mixture, msr = sf.read(i+'/mixture.wav')
        
        low = min([len(vocal), len(violin), len(mridangam), len(mixture)])
        
        violin = violin[:low]
        
        os.remove(i+'/violin.wav')
        sf.write(i+'/violin.wav', violin, visr)

100%|█████████████████████████████████████████| 161/161 [15:59<00:00,  5.96s/it]


In [11]:
dest_folder = "/home/rajesh/Desktop/Datasets/Saraga3stem/"
files = [dest_folder+i for i in os.listdir(dest_folder)]

for i in tqdm(files):
    file = os.listdir(i)
    if 'vocals.wav' in file and 'violin.wav' in file and 'mridangam.wav' in file and 'mixture.wav' in file:
        vocal, vsr = sf.read(i+'/vocals.wav')
        violin, visr = sf.read(i+'/violin.wav')
        mridangam, mrsr = sf.read(i+'/mridangam.wav')
        mixture, msr = sf.read(i+'/mixture.wav')
        
        low = min([len(vocal), len(violin), len(mridangam), len(mixture)])
        
        mridangam = mridangam[:low]
        
        os.remove(i+'/mridangam.wav')
        sf.write(i+'/mridangam.wav', mridangam, mrsr)

100%|█████████████████████████████████████████| 161/161 [15:43<00:00,  5.86s/it]


In [12]:
dest_folder = "/home/rajesh/Desktop/Datasets/Saraga3stem/"
files = [dest_folder+i for i in os.listdir(dest_folder)]

for i in tqdm(files):
    file = os.listdir(i)
    if 'vocals.wav' in file and 'violin.wav' in file and 'mridangam.wav' in file and 'mixture.wav' in file:
        vocal, vsr = sf.read(i+'/vocals.wav')
        violin, visr = sf.read(i+'/violin.wav')
        mridangam, mrsr = sf.read(i+'/mridangam.wav')
        mixture, msr = sf.read(i+'/mixture.wav')
        
        low = min([len(vocal), len(violin), len(mridangam), len(mixture)])
        
        vocal = vocal[:low]
        
        os.remove(i+'/vocals.wav')
        sf.write(i+'/vocals.wav', mridangam, mrsr)
            

100%|█████████████████████████████████████████| 161/161 [13:54<00:00,  5.18s/it]
